# Interactive Weather Data Visualization

This notebook provides comprehensive interactive visualizations of the Seattle weather dataset. We'll explore:
1. Temporal patterns
2. Weather distributions
3. Temperature analysis
4. Precipitation patterns
5. Wind analysis
6. Correlations
7. Seasonal trends
8. Weather type transitions
9. Monthly summaries
10. Multi-variable relationships

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Load and prepare the data
df = pd.read_csv('../seattle-weather.csv')
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['season'] = pd.cut(df['date'].dt.month, 
                     bins=[0,3,6,9,12], 
                     labels=['Winter', 'Spring', 'Summer', 'Fall'])

print("Data loaded successfully!")
print("\nDataset shape:", df.shape)
print("\nDate range:", df['date'].min().date(), "to", df['date'].max().date())

Data loaded successfully!

Dataset shape: (1461, 9)

Date range: 2012-01-01 to 2015-12-31


## 1. Temperature Trends and Patterns

In [2]:
# 1.1 Interactive Temperature Timeline
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=('Maximum Temperature', 'Minimum Temperature'))

fig.add_trace(
    go.Scatter(x=df['date'], y=df['temp_max'], mode='lines', name='Max Temp',
               line=dict(color='red')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df['date'], y=df['temp_min'], mode='lines', name='Min Temp',
               line=dict(color='blue')),
    row=2, col=1
)

fig.update_layout(height=800, title_text="Temperature Timeline",
                 showlegend=True)
fig.show()

# 1.2 Temperature Heatmap by Month and Year
monthly_temps = df.pivot_table(values='temp_max', 
                             index=df['date'].dt.year,
                             columns=df['date'].dt.month,
                             aggfunc='mean')

fig = go.Figure(data=go.Heatmap(
    z=monthly_temps,
    x=monthly_temps.columns,
    y=monthly_temps.index,
    colorscale='RdBu_r',
    text=np.round(monthly_temps, 1),
    texttemplate='%{text}°C',
    textfont={"size": 10},
    colorbar=dict(title='Temperature °C')
))

fig.update_layout(
    title='Monthly Average Maximum Temperature Heatmap',
    xaxis_title='Month',
    yaxis_title='Year'
)
fig.show()

# 1.3 Temperature Distribution by Weather Type
fig = go.Figure()

for weather_type in df['weather'].unique():
    temp_data = df[df['weather'] == weather_type]['temp_max']
    fig.add_trace(go.Violin(
        x=[weather_type] * len(temp_data),
        y=temp_data,
        name=weather_type,
        box_visible=True,
        meanline_visible=True
    ))

fig.update_layout(
    title='Temperature Distribution by Weather Type',
    xaxis_title='Weather',
    yaxis_title='Temperature (°C)',
    showlegend=False
)
fig.show()

## 2. Precipitation Analysis

In [3]:
# 2.1 Monthly Precipitation Patterns
monthly_precip = df.groupby(['year', 'month'])['precipitation'].mean().reset_index()
monthly_precip['date'] = pd.to_datetime(monthly_precip[['year', 'month']].assign(DAY=1))

fig = px.line(monthly_precip, x='date', y='precipitation',
              title='Monthly Precipitation Trends',
              line_shape='spline')
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Average Precipitation (mm)'
)
fig.show()

# 2.2 Precipitation Calendar Heatmap
precip_pivot = df.pivot_table(values='precipitation', 
                             index=df['date'].dt.year,
                             columns=df['date'].dt.month,
                             aggfunc='sum')

fig = go.Figure(data=go.Heatmap(
    z=precip_pivot,
    x=precip_pivot.columns,
    y=precip_pivot.index,
    colorscale='Blues',
    text=np.round(precip_pivot, 1),
    texttemplate='%{text}mm',
    textfont={"size": 10},
    colorbar=dict(title='Precipitation (mm)')
))

fig.update_layout(
    title='Monthly Precipitation Heatmap',
    xaxis_title='Month',
    yaxis_title='Year'
)
fig.show()

# 2.3 Precipitation vs Temperature Scatter
fig = px.scatter(df, x='temp_max', y='precipitation', 
                 color='weather', 
                 title='Precipitation vs Temperature by Weather Type',
                 labels={'temp_max': 'Maximum Temperature (°C)',
                        'precipitation': 'Precipitation (mm)',
                        'weather': 'Weather Type'},
                 hover_data=['date'])
fig.show()

# 2.4 Rain Probability by Month
monthly_rain_prob = df.groupby('month')['weather'].apply(
    lambda x: (x.isin(['rain', 'drizzle'])).mean() * 100
).reset_index()
monthly_rain_prob.columns = ['month', 'rain_probability']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthly_rain_prob['month'],
    y=monthly_rain_prob['rain_probability'],
    marker_color='skyblue'
))

fig.update_layout(
    title='Probability of Rain by Month',
    xaxis_title='Month',
    yaxis_title='Rain Probability (%)',
    showlegend=False
)
fig.show()

## 3. Weather Patterns and Seasonal Analysis

In [4]:
# 3.1 Weather Type Distribution by Season
seasonal_weather = pd.crosstab(df['season'], df['weather'], normalize='index') * 100

fig = go.Figure()
for weather_type in seasonal_weather.columns:
    fig.add_trace(go.Bar(
        name=weather_type,
        x=seasonal_weather.index,
        y=seasonal_weather[weather_type],
        text=np.round(seasonal_weather[weather_type], 1),
        textposition='auto',
    ))

fig.update_layout(
    barmode='stack',
    title='Weather Distribution by Season',
    xaxis_title='Season',
    yaxis_title='Percentage (%)'
)
fig.show()

# 3.2 Wind Speed Patterns
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=('Wind Speed Timeline', 'Wind Speed Distribution by Weather'))

# Wind Speed Timeline
fig.add_trace(
    go.Scatter(x=df['date'], y=df['wind'], mode='lines',
               name='Wind Speed', line=dict(color='darkblue')),
    row=1, col=1
)

# Wind Speed Distribution by Weather
for weather_type in df['weather'].unique():
    wind_data = df[df['weather'] == weather_type]['wind']
    fig.add_trace(
        go.Violin(x=[weather_type] * len(wind_data),
                 y=wind_data,
                 name=weather_type,
                 box_visible=True,
                 meanline_visible=True),
        row=2, col=1
    )

fig.update_layout(height=800, title_text="Wind Speed Analysis")
fig.show()

# 3.3 Weather Transition Sunburst
# Calculate weather transitions
df['next_weather'] = df['weather'].shift(-1)
transitions = pd.crosstab(df['weather'], df['next_weather'])

# Create sunburst data
labels = list(transitions.index) + [f"{i}->{j}" for i in transitions.index for j in transitions.columns]
parents = [''] * len(transitions.index) + [i for i in transitions.index for j in transitions.columns]
values = [transitions.sum(axis=1)[i] for i in transitions.index] + [transitions.values.flatten()]

fig = go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
))
fig.update_layout(title='Weather Pattern Transitions')
fig.show()

# 3.4 Multi-variable Weather Conditions
# Scale temperature for marker size to ensure all values are positive
df['temp_scaled'] = df['temp_max'] - df['temp_max'].min() + 1  # Add 1 to avoid zero sizes

fig = px.scatter_3d(df, x='temp_max', y='wind', z='precipitation',
                    color='weather', size='temp_scaled',
                    title='3D Weather Conditions Plot',
                    labels={'temp_max': 'Temperature (°C)',
                           'wind': 'Wind Speed (m/s)',
                           'precipitation': 'Precipitation (mm)'},
                    hover_data=['date'])
                    
# Update marker size range
fig.update_traces(marker=dict(sizeref=0.1))
fig.show()

## 4. Correlation and Advanced Analysis

In [5]:
# 4.1 Interactive Correlation Matrix
numerical_cols = ['temp_max', 'temp_min', 'precipitation', 'wind']
corr_matrix = df[numerical_cols].corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    text=np.round(corr_matrix, 2),
    texttemplate='%{text}',
    textfont={"size": 12},
    colorscale='RdBu',
    zmid=0
))

fig.update_layout(
    title='Correlation Matrix of Weather Variables',
    width=700,
    height=700
)
fig.show()

# 4.2 Weather Variables Parallel Coordinates
# Create a numerical encoding for weather types
weather_encoding = {weather: i for i, weather in enumerate(df['weather'].unique())}
df['weather_code'] = df['weather'].map(weather_encoding)

fig = px.parallel_coordinates(df, 
                            dimensions=['temp_max', 'temp_min', 'precipitation', 'wind'],
                            color='weather_code',
                            color_continuous_scale=px.colors.qualitative.Set3,
                            title='Parallel Coordinates Plot of Weather Variables')

# Update color axis to show weather types
fig.update_layout(
    coloraxis_colorbar=dict(
        title='Weather Type',
        ticktext=list(weather_encoding.keys()),
        tickvals=list(weather_encoding.values()),
    )
)
fig.show()

# 4.3 Temperature-Precipitation Relationship by Season
fig = px.scatter(df, x='temp_max', y='precipitation',
                 color='season', size='wind',
                 facet_col='season', 
                 title='Temperature-Precipitation Relationship by Season',
                 labels={'temp_max': 'Maximum Temperature (°C)',
                        'precipitation': 'Precipitation (mm)',
                        'wind': 'Wind Speed (m/s)'},
                 hover_data=['date', 'weather'])
fig.show()

# 4.4 Monthly Weather Type Distribution Animation
monthly_weather_dist = pd.crosstab(df['month'], df['weather'], normalize='index') * 100

fig = px.bar(monthly_weather_dist.reset_index().melt(id_vars='month'),
             x='weather', y='value', color='weather',
             animation_frame='month',
             range_y=[0,100],
             title='Monthly Weather Type Distribution',
             labels={'value': 'Percentage (%)', 'weather': 'Weather Type'})

fig.update_layout(
    xaxis_title='Weather Type',
    yaxis_title='Percentage (%)'
)
fig.show()